In [ ]:
# Initialisation
# Import packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

# Import ML packages
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input

# Import Google Drive to allow access to data
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Check TensorFlow Version
print(tf.__version__)

# Initialise variable that defines the location of dataset
dataLoc = '[REDACTED]'

# Read in dataset
df = pd.read_csv(dataLoc)

# Change the frame variable from numerical integer to categorical
df['Frame'] = df['Frame'].astype('category')
df['BattCells'] = df['BattCells'].astype('category')

# Drop Weight and Remaining Load columns
df = df.drop(['Weight','RemLoad'], axis=1)
df = df.drop(columns = ['NLC','NLV','LimCurrent','InternalR','Blades','ESCCells'])

# Separate categorical variables into a boolean definition
# df = pd.get_dummies(df, columns = ['Frame'],prefix = '', prefix_sep = '')
# df = pd.get_dummies(df, columns = ['BattCells'],prefix = '', prefix_sep = '')

# Visual check of the dataframe to ensure that it is the same as what we are after
df.tail()

# Set the training and test datasets
train_data = df.sample(frac = 0.8, random_state = 0)
test_data = df.drop(train_data.index)

# Check Data Types
df.dtypes

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
2.14.0


Frame            category
KV                  int64
motDiam             int64
propDiam            int64
propPitch           int64
ESCCurrent          int64
BattCells        category
BattCap             int64
BattDischarge       int64
Endurance         float64
Range             float64
MaxSpeed          float64
MTOW              float64
dtype: object

In [ ]:
# Define a function to extract output variables
def outputConfigure(data):
    outFrame = data.pop('Frame')
    outFrame = np.array(outFrame)
    outKV = data.pop('KV')
    outKV = np.array(outKV)
    outMotDiam = data.pop('motDiam')
    outMotDiam = np.array(outMotDiam)
    outPropDiam = data.pop('propDiam')
    outPropDiam = np.array(outPropDiam)
    outPropPitch = data.pop('propPitch')
    outPropPitch = np.array(outPropPitch)
    outESCCurrent = data.pop('ESCCurrent')
    outESCCurrent = np.array(outESCCurrent)
    outBattCells = data.pop('BattCells')
    outBattCells = np.array(outBattCells)
    outBattCap = data.pop('BattCap')
    outBattCap = np.array(outBattCap)
    outBattDischarge = data.pop('BattDischarge')
    outBattDischage = np.array(outBattDischarge)

    return outFrame, outKV, outMotDiam, outPropDiam, outPropPitch, outESCCurrent, outBattCells, outBattCap, outBattDischarge

# Define a function to normalise input data
def normalise(var):
    return (var - trainStats['mean'])/trainStats['std']

In [ ]:
# Load data for the model
trainFeatures = train_data.copy()
testFeatures = test_data.copy()

# Drop input variables
trainFeatures = trainFeatures.drop(columns = ['MTOW','Range','Endurance','MaxSpeed'])
testFeatures = testFeatures.drop(columns = ['MTOW','Range','Endurance','MaxSpeed'])

# Remove all non-input variables
trainFeaturesSubs = train_data[['MTOW','Range','Endurance','MaxSpeed']]
testFeaturesSubs = test_data[['MTOW','Range','Endurance','MaxSpeed']]

In [ ]:
# Get data metrics
trainStats = trainFeaturesSubs.describe()
trainStats = trainStats.T
print(trainStats)

             count       mean       std   min    25%    50%    75%    max
MTOW       41317.0   1.528431  0.683310  0.40   1.02   1.42   1.92   3.87
Range      41317.0   5.058426  2.125099  0.00   3.54   4.67   6.22  16.35
Endurance  41317.0   9.354633  4.890360  1.42   5.68   8.46  12.00  38.08
MaxSpeed   41317.0  18.572108  6.962373  0.00  13.30  16.90  23.40  40.70


In [ ]:
# Normalise training and test data
normTrainIn = np.array(normalise(trainFeaturesSubs))
normTestIn = np.array(normalise(testFeaturesSubs))

# Acquire Output Data for train and test data
trainOut = outputConfigure(trainFeatures)
testOut = outputConfigure(testFeatures)

In [ ]:
# Define the model construction and layer use

def modelBuild():

    inputLayer = Input(shape=(len(trainFeaturesSubs.columns),))
    # Run Layer and define output for Frame Size (Cat)
    FSOut = Dense(9, activation = 'softmax', name = 'FSOut')(inputLayer)

    # Run Layers for KV (Reg)
    denseKV1 = Dense(units = '64', activation = 'relu')(inputLayer)
    denseKV2 = Dense(units = '64', activation = 'relu')(denseKV1)
    denseKV3 = Dense(units = '64', activation = 'relu')(denseKV2)
    # Define Output for KV
    KVOut = Dense(units = '1', name = 'KVOut')(denseKV3)

    # Run Layers for Motor Diameter (Reg)
    denseMD1 = Dense(units = '64', activation = 'relu')(inputLayer)
    denseMD2 = Dense(units = '64', activation = 'relu')(denseMD1)
    # Define Output for KV
    MDOut = Dense(units = '1', name = 'MDOut')(denseMD2)

    # Run Layers for Propeller Diameter (Reg)
    densePD1 = Dense(units = '64', activation = 'relu')(inputLayer)
    densePD2 = Dense(units = '64', activation = 'relu')(densePD1)
    densePD3 = Dense(units = '64', activation = 'relu')(densePD2)
    # Define Output for KV
    PDOut = Dense(units = '1', name = 'PDOut')(densePD3)

    # Run Layers for Propeller Pitch (Reg)
    densePP1 = Dense(units = '64', activation = 'relu')(inputLayer)
    densePP2 = Dense(units = '64', activation = 'relu')(densePP1)
    densePP3 = Dense(units = '64', activation = 'relu')(densePP2)
    # Define Output for KV
    PPOut = Dense(units = '1', name = 'PPOut')(densePP3)

    # Run Layers for ESC Current (Reg)
    denseESCC1 = Dense(units = '64', activation = 'relu')(inputLayer)
    denseESCC2 = Dense(units = '64', activation = 'relu')(denseESCC1)
    denseESCC3 = Dense(units = '64', activation = 'relu')(denseESCC2)
    # Define Output for KV
    ESCCOut = Dense(units = '1', name = 'ESCCOut')(denseESCC3)

    # Run Layers for Battery Cells (Cat)
    BCeOut = Dense(3, activation = 'softmax', name = 'BCeOut')(inputLayer)

    # Run Layers for Battery Capacity (Reg)
    denseBCa1 = Dense(units = '64', activation = 'relu')(inputLayer)
    denseBCa2 = Dense(units = '64', activation = 'relu')(denseBCa1)
    denseBCa3 = Dense(units = '64', activation = 'relu')(denseBCa2)
    # Define Output for KV
    BCaOut = Dense(units = '1', name = 'BCaOut')(denseBCa3)

    # Run Layers for Battery Discharge (Reg)
    denseBD1 = Dense(units = '64', activation = 'relu')(inputLayer)
    denseBD2 = Dense(units = '64', activation = 'relu')(denseBD1)
    denseBD3 = Dense(units = '64', activation = 'relu')(denseBD2)
    # Define Output for KV
    BDOut = Dense(units = '1', name = 'BDOut')(denseBD3)

    # Define model
    model = Model(inputs = inputLayer, outputs = [FSOut, KVOut, MDOut, PDOut, PPOut, ESCCOut, BCeOut, BCaOut, BDOut])

    return model

In [ ]:
# Run Model Building function defined above
model = modelBuild()

# Select an optimiser
optimiser = tf.keras.optimizers.Adam(0.001)

# Compile DNN Model and define outputs and error values
model.compile(optimizer = optimiser,
              loss = {'FSOut': 'mae', 'KVOut': 'mae', 'MDOut': 'mae','PDOut': 'mae','PPOut': 'mae','ESCCOut': 'mae', 'BCeOut': 'mae', 'BCaOut': 'mae', 'BDOut': 'mae'},
              metrics = {'FSOut': tf.keras.metrics.MeanAbsoluteError(),
                         'KVOut': tf.keras.metrics.MeanAbsoluteError(),
                         'MDOut': tf.keras.metrics.MeanAbsoluteError(),
                         'PDOut': tf.keras.metrics.MeanAbsoluteError(),
                         'PPOut': tf.keras.metrics.MeanAbsoluteError(),
                         'ESCCOut': tf.keras.metrics.MeanAbsoluteError(),
                         'BCeOut': tf.keras.metrics.MeanAbsoluteError(),
                         'BCaOut': tf.keras.metrics.MeanAbsoluteError(),
                         'BDOut': tf.keras.metrics.MeanAbsoluteError()})

In [ ]:
# Train the model for 100 epochs, similar to other scripts
history = model.fit(normTrainIn,trainOut,
                    validation_split = 0.2,
                    epochs = 100)

Epoch 1/100
1033/1033 [==============================] - 14s 7ms/step - loss: 1412.0823 - FSOut_loss: 307.5799 - KVOut_loss: 551.6444 - MDOut_loss: 3.1610 - PDOut_loss: 0.4111 - PPOut_loss: 0.9566 - ESCCOut_loss: 5.7851 - BCeOut_loss: 2.3281 - BCaOut_loss: 534.6259 - BDOut_loss: 5.5888 - FSOut_mean_absolute_error: 307.5799 - KVOut_mean_absolute_error: 551.6444 - MDOut_mean_absolute_error: 3.1610 - PDOut_mean_absolute_error: 0.4111 - PPOut_mean_absolute_error: 0.9566 - ESCCOut_mean_absolute_error: 5.7851 - BCeOut_mean_absolute_error: 2.3281 - BCaOut_mean_absolute_error: 534.6259 - BDOut_mean_absolute_error: 5.5888 - val_loss: 771.1236 - val_FSOut_loss: 309.4252 - val_KVOut_loss: 220.2586 - val_MDOut_loss: 1.5868 - val_PDOut_loss: 0.1293 - val_PPOut_loss: 0.8197 - val_ESCCOut_loss: 4.8862 - val_BCeOut_loss: 2.3357 - val_BCaOut_loss: 226.8252 - val_BDOut_loss: 4.8576 - val_FSOut_mean_absolute_error: 309.4252 - val_KVOut_mean_absolute_error: 220.2586 - val_MDOut_mean_absolute_error: 1.5868

In [ ]:

# Define Test Prediction 
testPred = model.predict(normTestIn)

# Load Predicted Input
dataLoc2 = '[REDACTED]'
# Read in dataset
predictdf = pd.read_csv(dataLoc2)

# Normalise data
predictNorm = np.array(normalise(predictdf))

# Make Prediction
predictList = model.predict(predictNorm)

323/323 [==============================] - 1s 3ms/step
[array([[0.01759833, 0.12687005, 0.1965328 , ..., 0.01549755, 0.45136607,
        0.01101177],
       [0.0336727 , 0.1432136 , 0.18272   , ..., 0.02632677, 0.40910935,
        0.01314119],
       [0.03356735, 0.14596044, 0.18472522, ..., 0.02604037, 0.39975753,
        0.01352021],
       ...,
       [0.09688378, 0.04545205, 0.03738395, ..., 0.08455106, 0.01247144,
        0.4216633 ],
       [0.09769194, 0.02982935, 0.03309954, ..., 0.13368201, 0.02057289,
        0.4188076 ],
       [0.07920932, 0.03450161, 0.04338666, ..., 0.12419632, 0.03079072,
        0.38915905]], dtype=float32), array([[3854.3394 ],
       [3650.675  ],
       [3582.4983 ],
       ...,
       [ 617.3662 ],
       [ 957.44086],
       [ 923.39856]], dtype=float32), array([[ 9.808435],
       [ 9.808717],
       [ 9.812583],
       ...,
       [27.782389],
       [27.688576],
       [27.8787  ]], dtype=float32), array([[ 3.0010335],
       [ 2.9989126],
     